```
What to do
공유 자전거의 위치데이터를 받을 수 없기에, 임의로 설정을 해야함

----------------------------------------------------------------------------------
Problem 1) 어떤 정류장에 할당을 해줄것인가?
sol 1) 사람들이 많이 타고내리는(승하차량이 많은) 정류장에 할당을 해주자!
sol fail) 특정 지역에 상위랭킹 정류장이 밀집됨 (ex.부평역, 구월동 ...)

sol clear) Clustering Algorithm을 통해 가까운 지역에 있는 정류장은 한 군집으로 묶고, 군집 별 승하차량으로 랭킹을 매겨주기로함. 군집은 2000개, 군집중 상위 100개만 선정. 2000개와 100개는 각각 변경가능
           
----------------------------------------------------------------------------------
Problem 2) 선정된 군집에 어떻게 자전거를 할당해 줄 것인가?

sol clear) 선정된 군집에 할당된 정류장의 중심점을 중심으로 다변량(여기선 x,y 위경도) 정규분포를 적용하여 각 군집당 100개의 자전거를 랜덤하게 할당
       
```


In [20]:
"""
import required libraries
"""
import pickle
import folium
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [3]:
"""
Transportation card data necessary for calculating passengers 
between stops for a period of 3 weeks
"""
root1 = 'Dropbox/File_Streaming/201807TripChain/'
root2 = 'Dropbox/File_Streaming/201811TripChain/'
root3 = 'Dropbox/File_Streaming/201909TripChain/'
filelist_1807 = [root1+x for x in os.listdir(root1)]
filelist_1811 = [root2+x for x in os.listdir(root2)]
filelist_1909 = [root3+x for x in os.listdir(root3)]

filelist = filelist_1807+filelist_1811+filelist_1909
filelist

['Dropbox/File_Streaming/201807TripChain/DI_TBTRDD050_20180701.csv',
 'Dropbox/File_Streaming/201807TripChain/DI_TBTRDD050_20180702.csv',
 'Dropbox/File_Streaming/201807TripChain/DI_TBTRDD050_20180703.csv',
 'Dropbox/File_Streaming/201807TripChain/DI_TBTRDD050_20180704.csv',
 'Dropbox/File_Streaming/201807TripChain/DI_TBTRDD050_20180705.csv',
 'Dropbox/File_Streaming/201807TripChain/DI_TBTRDD050_20180706.csv',
 'Dropbox/File_Streaming/201807TripChain/DI_TBTRDD050_20180707.csv',
 'Dropbox/File_Streaming/201811TripChain/DI_TBTRDD050_20181101.csv',
 'Dropbox/File_Streaming/201811TripChain/DI_TBTRDD050_20181102.csv',
 'Dropbox/File_Streaming/201811TripChain/DI_TBTRDD050_20181103.csv',
 'Dropbox/File_Streaming/201811TripChain/DI_TBTRDD050_20181104.csv',
 'Dropbox/File_Streaming/201811TripChain/DI_TBTRDD050_20181105.csv',
 'Dropbox/File_Streaming/201811TripChain/DI_TBTRDD050_20181106.csv',
 'Dropbox/File_Streaming/201811TripChain/DI_TBTRDD050_20181107.csv',
 'Dropbox/File_Streaming/201909Tri

In [4]:
"""
station status data
"""
stn_status_df = pd.read_excel('data/excel_station_status.xlsx',
                               usecols=['정류소 명','정류소 번호','ISC 정류소 ID','위도','경도'],
                               dtype=str)
stn_status_df.head()

,정류소 명,정류소 번호,ISC 정류소 ID,위도,경도
0,(구)시민회관사거리,37319,2802757,126.68018830433286,37.457360395358165
1,(구)시민회관사거리,37326,2802755,126.67992703811578,37.45776642315442
2,(구)신현주공,42228,2800619,126.67339452968905,37.52184277702059
3,(구)현대백화점,40158,2801688,126.71126751729724,37.490948333960574
4,(주)경동세라믹스,89146,2806735,126.61697759175779,37.559185515339


In [6]:
data = {x:0 for x in stn_status_df['ISC 정류소 ID']} 
## data = {'isc_id1':0,'isc_id2':0 ... }

"""
data = {'isc_id1' : 99, 'isc_id2': 167 ... }

It will take long time....
"""
for file in filelist:
    print(f"{file.split('/')[-1]} start....")
    input_file = open(file,'rb')
    line = input_file.readline()
    while True:
        line = input_file.readline().decode()
        if not line:break
        line = line.split(',')
        if line[0] =='':continue
        num_transfer = int(line[2])
        passenger = int(line[4])
        try:
            for n in range(num_transfer+1):
                o = line[n+36][1:-1]
                if o in data:
                    data[o] += passenger
                d = line[n+41][1:-1]
                if d in data:
                    data[d] += passenger
        except ValueError:
            continue
    print(f"{file.split('/')[-1]} completed....")

DI_TBTRDD050_20180701.csv start....
DI_TBTRDD050_20180701.csv completed....
DI_TBTRDD050_20180702.csv start....
DI_TBTRDD050_20180702.csv completed....
DI_TBTRDD050_20180703.csv start....
DI_TBTRDD050_20180703.csv completed....
DI_TBTRDD050_20180704.csv start....
DI_TBTRDD050_20180704.csv completed....
DI_TBTRDD050_20180705.csv start....
DI_TBTRDD050_20180705.csv completed....
DI_TBTRDD050_20180706.csv start....
DI_TBTRDD050_20180706.csv completed....
DI_TBTRDD050_20180707.csv start....
DI_TBTRDD050_20180707.csv completed....
DI_TBTRDD050_20181101.csv start....
DI_TBTRDD050_20181101.csv completed....
DI_TBTRDD050_20181102.csv start....
DI_TBTRDD050_20181102.csv completed....
DI_TBTRDD050_20181103.csv start....
DI_TBTRDD050_20181103.csv completed....
DI_TBTRDD050_20181104.csv start....
DI_TBTRDD050_20181104.csv completed....
DI_TBTRDD050_20181105.csv start....
DI_TBTRDD050_20181105.csv completed....
DI_TBTRDD050_20181106.csv start....
DI_TBTRDD050_20181106.csv completed....
DI_TBTRDD050

In [10]:
"""import kmeans for clustering"""
from sklearn.cluster import KMeans

In [11]:
dropped_stn_df = stn_status_df.dropna()
mapping = {key:dropped_stn_df.iloc[key,2] for key in range(len(dropped_stn_df))} ## isc id와 일반 id mapping
point_data = dropped_stn_df.values[:,-2:].astype(float) ## 정류소 별 위경도(kmeans의 input data)
point_data

array([[126.6801883 ,  37.4573604 ],
       [126.67992704,  37.45776642],
       [126.67339453,  37.52184278],
       ...,
       [126.72966956,  37.49224732],
       [126.73000115,  37.49242456],
       [126.67816399,  37.41441869]])

In [12]:
kmeans = KMeans(n_clusters=2000,random_state=42).fit(point_data)

In [13]:
kmeans.labels_

array([1322,  444, 1834, ...,  587,  587,  507])

In [39]:
output = {label:{'stn_list':[],'passenger':0,'lonlat':0} for label in kmeans.labels_}

"""
output = {1322: {'stn_list': ['2802757','2802733','2802731','2802738','2802739','2802732'],
                 'passenger': 267690,
                 'lonlat': (126.681556331182, 37.45866754957071)},
                 
           444: {'stn_list': ['2802755', '2802643', '2802748', '2802617', '2802751'],
                 'passenger': 153286,
                 'lonlat': (126.67828600880651, 37.4584561742317)}}
"""

for i,x in enumerate(kmeans.labels_):
    isc_id = mapping[i]
    output[x]['stn_list'].append(isc_id)
    output[x]['passenger']+=data[isc_id]
for i in output:
    stn_list = output[i]['stn_list']
    sum_lon,sum_lat = 0,0
    for stn in stn_list:
        sum_lon+=float(dropped_stn_df[dropped_stn_df['ISC 정류소 ID']==stn]['위도'].values[0])
        sum_lat+=float(dropped_stn_df[dropped_stn_df['ISC 정류소 ID']==stn]['경도'].values[0])
    output[i]['lonlat'] = sum_lon/len(stn_list),sum_lat/len(stn_list)



In [29]:
"""
군집별 위경도를 key, 군집에 해당하는 정류소들의 승하차량 총합 value
"""
result = {output[x]['lonlat']:output[x]['passenger'] for x in output}
result

{(126.681556331182, 37.45866754957071): 267690,
 (126.67828600880651, 37.4584561742317): 153286,
 (126.67353805637299, 37.52195918309027): 27271,
 (126.71162641241109, 37.49078385084421): 77867,
 (126.6170385531781, 37.55903662511193): 525,
 (126.66698672428655, 37.499834413613385): 15177,
 (126.70704883647811, 37.41650024406452): 23343,
 (126.69423054715158, 37.399976932406666): 1725,
 (126.7028555182685, 37.41847551185656): 3533,
 (126.69895221388856, 37.41860413534931): 1863,
 (126.6847964784744, 37.47277274400065): 2412,
 (126.6196667645734, 37.55976397458218): 836,
 (126.69913972736495, 37.393939585576334): 12898,
 (126.6571377738278, 37.61480965316267): 10674,
 (126.66010222029107, 37.613520250403454): 26985,
 (126.70459628920636, 37.48879925950423): 45188,
 (126.66797828893273, 37.51278320182024): 31593,
 (126.65071536465341, 37.49147616804343): 5521,
 (126.72648769003463, 37.46439979376434): 9718,
 (126.74547693187908, 37.481753433212525): 10587,
 (126.74783526596559, 37.478380

In [33]:
import pickle
ranked_spot = sorted(result, key=result.get, reverse=True)
pickle.dump(ranked_spot,open('ranked_spot.txt','wb'))

In [34]:
ranked_spot

[(126.70210127788673, 37.44236898171395),
 (126.68079643599089, 37.4638666121099),
 (126.72523718400436, 37.49252577968656),
 (126.70341181448009, 37.471744207061526),
 (126.7090045189723, 37.452450417729274),
 (126.71975317720391, 37.50777452266914),
 (126.75292733005833, 37.48727302901877),
 (126.67500631416067, 37.52510357716665),
 (126.681556331182, 37.45866754957071),
 (126.72332380697551, 37.49598469282277),
 (126.64969559374222, 37.44973981995151),
 (126.68912897226546, 37.45849890950243),
 (126.63177486419096, 37.476160639949235),
 (126.72100687163899, 37.49127276797505),
 (126.70802730870466, 37.48368912901649),
 (126.68079244330725, 37.448100968775464),
 (126.63113919447585, 37.47455345296954),
 (126.62883823850069, 37.47014042725539),
 (126.65631588408182, 37.465956634743584),
 (126.73128022605978, 37.507135711183324),
 (126.721530739056, 37.51715275040913),
 (126.70707856824521, 37.46694479723529),
 (126.67335640392021, 37.39858194778008),
 (126.73455883528241, 37.460776374

In [35]:
ranked_spot = pickle.load(open('ranked_spot.txt','rb'))
ranked_spot

[(126.70210127788673, 37.44236898171395),
 (126.68079643599089, 37.4638666121099),
 (126.72523718400436, 37.49252577968656),
 (126.70341181448009, 37.471744207061526),
 (126.7090045189723, 37.452450417729274),
 (126.71975317720391, 37.50777452266914),
 (126.75292733005833, 37.48727302901877),
 (126.67500631416067, 37.52510357716665),
 (126.681556331182, 37.45866754957071),
 (126.72332380697551, 37.49598469282277),
 (126.64969559374222, 37.44973981995151),
 (126.68912897226546, 37.45849890950243),
 (126.63177486419096, 37.476160639949235),
 (126.72100687163899, 37.49127276797505),
 (126.70802730870466, 37.48368912901649),
 (126.68079244330725, 37.448100968775464),
 (126.63113919447585, 37.47455345296954),
 (126.62883823850069, 37.47014042725539),
 (126.65631588408182, 37.465956634743584),
 (126.73128022605978, 37.507135711183324),
 (126.721530739056, 37.51715275040913),
 (126.70707856824521, 37.46694479723529),
 (126.67335640392021, 37.39858194778008),
 (126.73455883528241, 37.460776374

In [36]:
m = folium.Map(location=ranked_spot[0][::-1],zoom_start=10)
for stn in ranked_spot[:100]:
    bike_lonlats = np.random.multivariate_normal([stn[1],stn[0]],[[0.0000001,0.00001],[0.00001,0.00001]],10)
    for bike_lonlat in bike_lonlats:
        folium.Circle(
          location=bike_lonlat,
            radius=5
        ).add_to(m)
m

C:\Users\skatl\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  This is separate from the ipykernel package so we can avoid doing imports until
